# Data Extraction and Web Scraping

We will use a real-world example to demonstrate the concepts of data extraction via REST API and GraphQL. The APIs we will be using are the [GitHub REST API](https://docs.github.com/en/rest) and [GitHub GraphQL API](https://docs.github.com/en/graphql).

They are both APIs provided by GitHub to access data on their platform. The GraphQL API is a newer API. You can access data such as user profiles, repositories, issues, pull requests, releases etc. from GitHub.

We will use the following libraries for this lesson:

- `requests` for making HTTP requests
- `json` for parsing JSON responses
- `pandas` and `numpy` for data manipulation
- `sqlalchemy` for writing data to database (DuckDB)
- `beautifulsoup4` for web scraping

## REST API

Usually, before we can make a request to an API, we need to register for an API key. This is for the API provider to track usage, prevent abuse and to authenticate users. For more information on how to authenticate with GitHub API, refer to the [GitHub API docs](https://docs.github.com/en/rest/overview/resources-in-the-rest-api?apiVersion=2022-11-28#authentication). 

For this lesson, you will need to a personal access token (API key). If you do not have an API key, you can create a new one by folllowing the instructions in the official GitHub documentation [here](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens#creating-a-fine-grained-personal-access-token).

We will be retrieving data from some popular repositories on GitHub.

In [1]:
import requests

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the GitHub token
access_token = os.getenv('GITHUB_TOKEN')

Github API endpoints are in the format 
```
https://api.github.com/repos/{owner}/{repo}
```
and appended with the resource you want to access.

For example, to access the `issues` resource, the endpoint is
```
https://api.github.com/repos/{owner}/{repo}/issues
```

---

Here are some common endpoints:

| Resource | Endpoint |
| --- | --- |
| Issues | `/issues` |
| Pull Requests | `/pulls` |
| Commits | `/commits` |
| Contributors | `/contributors` |
| Languages | `/languages` |
| Releases | `/releases` |
| Tags | `/tags` |
| Branches | `/branches` |
| Forks | `/forks` |
| Stargazers | `/stargazers` |
| Subscribers | `/subscribers` |
| Subscription | `/subscription` |

Here are the HTTP verbs for the Github API:

| Verb | Description |
| --- | --- |
| GET | Used for retrieving resources. |
| POST | Used for creating resources. |
| PATCH | Used for updating resources with partial JSON data. For instance, an Issue resource has title and body attributes. A PATCH request may accept one or more of the attributes to update the resource. |
| PUT | Used for replacing resources or collections. For PUT requests with no body attribute, be sure to set the Content-Length header to zero. |
| DELETE | Used for deleting resources. |

We will only be using the GET verb for this project, 


`Pandas` is an open-source library, its code repository is hosted on GitHub. You can access the repository at [https://github.com/pandas-dev/pandas](https://github.com/pandas-dev/pandas).

Let's try to extract the past `releases` (library versions) from the the repository. The API endpoint is `https://api.github.com/repos/pandas-dev/pandas/releases`.

The access token needs to be passed in the `Authorization` header. The token is a personal access token (PAT) which you can generate on GitHub. The token is used to authenticate you as a user and to authorize you to access the repository.

It is also recommended to pass in the following headers:

- `Accept: application/vnd.github+json` - to specify the preferred format of the response
- `X-GitHub-Api-Version:2022-11-28` - to specify the version of the API to use

---

If we use a `GET` request to access the endpoint, we will get a list of releases in JSON format (default limit of _30 results_ per page). More info on the `releases` resource [here](https://docs.github.com/en/rest/releases/releases?apiVersion=2022-11-28).

In [3]:
response = requests.get("https://api.github.com/repos/pandas-dev/pandas/releases", 
                        headers={"Accept": "application/vnd.github+json", "Authorization": f"Bearer {access_token}"})

We can check if the response is successful by checking the status code of the response. If the status code is 200, then the response is successful.

In [4]:
response.status_code

200

Use the `.json()` method to return the JSON format of the response object.

In [5]:
releases_page_1 = response.json()

In [6]:
type(releases_page_1)

list

In [7]:
len(releases_page_1)

30

In [8]:
releases_page_1[0]

{'url': 'https://api.github.com/repos/pandas-dev/pandas/releases/230570685',
 'assets_url': 'https://api.github.com/repos/pandas-dev/pandas/releases/230570685/assets',
 'upload_url': 'https://uploads.github.com/repos/pandas-dev/pandas/releases/230570685/assets{?name,label}',
 'html_url': 'https://github.com/pandas-dev/pandas/releases/tag/v2.3.1',
 'id': 230570685,
 'author': {'login': 'jorisvandenbossche',
  'id': 1020496,
  'node_id': 'MDQ6VXNlcjEwMjA0OTY=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/1020496?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/jorisvandenbossche',
  'html_url': 'https://github.com/jorisvandenbossche',
  'followers_url': 'https://api.github.com/users/jorisvandenbossche/followers',
  'following_url': 'https://api.github.com/users/jorisvandenbossche/following{/other_user}',
  'gists_url': 'https://api.github.com/users/jorisvandenbossche/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/jorisvandenbossche/starred{

In [9]:
releases_page_1[0]['name']

'Pandas 2.3.1'

In [10]:
releases_page_1[0]['published_at']

'2025-07-07T19:12:32Z'

The latest version of `pandas` is the one shown above. Hence, the list is sorted from most recent to oldest.

While the last release on the list is:

In [11]:
releases_page_1[-1]['name']

'Pandas 1.4.0rc0'

In [12]:
releases_page_1[-1]['published_at']

'2022-01-06T11:02:19Z'

Remember the default results per page is `30`. You can increase the number of results per page to a maximum of `100` by using the `per_page` query parameter.

In [13]:
response = requests.get("https://api.github.com/repos/pandas-dev/pandas/releases?per_page=100", 
                        headers={"Accept": "application/vnd.github+json", "Authorization": f"Bearer{access_token}"})

In [14]:
releases_page_1_100 = response.json()

In [15]:
len(releases_page_1_100)

100

In [16]:
releases_page_1_100[-1]

{'url': 'https://api.github.com/repos/pandas-dev/pandas/releases/1047005',
 'assets_url': 'https://api.github.com/repos/pandas-dev/pandas/releases/1047005/assets',
 'upload_url': 'https://uploads.github.com/repos/pandas-dev/pandas/releases/1047005/assets{?name,label}',
 'html_url': 'https://github.com/pandas-dev/pandas/releases/tag/v0.16.0rc1',
 'id': 1047005,
 'author': {'login': 'jreback',
  'id': 953992,
  'node_id': 'MDQ6VXNlcjk1Mzk5Mg==',
  'avatar_url': 'https://avatars.githubusercontent.com/u/953992?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/jreback',
  'html_url': 'https://github.com/jreback',
  'followers_url': 'https://api.github.com/users/jreback/followers',
  'following_url': 'https://api.github.com/users/jreback/following{/other_user}',
  'gists_url': 'https://api.github.com/users/jreback/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/jreback/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/jreback/subsc

The default page is `1`, you can get the 2nd page by passing the `page` query parameter.

In [17]:
response = requests.get("https://api.github.com/repos/pandas-dev/pandas/releases?per_page=100&page=2", 
                        headers={"Accept": "application/vnd.github+json", "Authorization": f"Bearer{access_token}"})

In [18]:
releases_page_2_100 = response.json()

In [19]:
len(releases_page_2_100)

11


We're interested in the following fields and want to save them into a dataframe:

- `tag_name`: The version name/number of the release.
- `published_at`: The date and time when the release was published.
- `body`: The release notes.

In [20]:
import pandas as pd

In [21]:
releases_with_essential_fields = [{"version": release["tag_name"], 
                                "published_at": release["published_at"], 
                                "summary": release["body"]} for release in releases_page_1_100]

In [22]:
pandas_releases = pd.DataFrame(releases_with_essential_fields)

pandas_releases

,version,published_at,summary
0,v2.3.1,2025-07-07T19:12:32Z,We are pleased to announce the release of pand...
1,v2.3.0,2025-06-05T03:17:32Z,We are pleased to announce the release of pand...
2,v2.2.3,2024-09-20T13:10:41Z,We are pleased to announce the release of pand...
3,v2.2.2,2024-04-10T19:47:02Z,We are pleased to announce the release of pand...
4,v2.2.1,2024-02-23T15:29:27Z,We are pleased to announce the release of pand...
...,...,...,...
95,v0.17.0rc1,2015-09-11T16:52:08Z,**RELEASE CANDIDATE 1**\n\nThis is a major rel...
96,v0.16.2,2015-06-13T10:49:42Z,This is a minor bug-fix release from 0.16.1 an...
97,v0.16.1,2015-05-11T01:19:26Z,This is a minor bug-fix release from 0.16.0 an...
98,v0.16.0,2015-03-22T13:42:47Z,This is a major release from 0.15.2 and includ...


In [23]:
# convert `published_at` to the correct datetime format
pandas_releases['published_at'] = pd.to_datetime(pandas_releases['published_at'])

pandas_releases

,version,published_at,summary
0,v2.3.1,2025-07-07 19:12:32+00:00,We are pleased to announce the release of pand...
1,v2.3.0,2025-06-05 03:17:32+00:00,We are pleased to announce the release of pand...
2,v2.2.3,2024-09-20 13:10:41+00:00,We are pleased to announce the release of pand...
3,v2.2.2,2024-04-10 19:47:02+00:00,We are pleased to announce the release of pand...
4,v2.2.1,2024-02-23 15:29:27+00:00,We are pleased to announce the release of pand...
...,...,...,...
95,v0.17.0rc1,2015-09-11 16:52:08+00:00,**RELEASE CANDIDATE 1**\n\nThis is a major rel...
96,v0.16.2,2015-06-13 10:49:42+00:00,This is a minor bug-fix release from 0.16.1 an...
97,v0.16.1,2015-05-11 01:19:26+00:00,This is a minor bug-fix release from 0.16.0 an...
98,v0.16.0,2015-03-22 13:42:47+00:00,This is a major release from 0.15.2 and includ...


> What is the average interval (duration or differences in time) for pandas' releases (in no of days)?
>
> Hint 1: Sort the dataframe by `published_at` in ascending order.
> 
> Hint 2: Use [diff()](https://pandas.pydata.org/docs/reference/api/pandas.Series.diff.html) to calculate the time deltas between each `published_at`.
>
> Refer to [Time deltas](https://pandas.pydata.org/pandas-docs/stable/user_guide/timedeltas.html#time-deltas) for more info.

In [24]:
# Sort the dataframe by published_at in ascending order (oldest first)
pandas_releases_sorted = pandas_releases.sort_values('published_at', ascending=True)

# Calculate the time differences between consecutive releases
time_diffs = pandas_releases_sorted['published_at'].diff()

# Convert to days and calculate the average
# The diff() function returns timedelta objects, we extract days
average_interval_days = time_diffs.dt.days.mean()

print(f"Average interval between pandas releases: {average_interval_days:.1f} days")

# Show some statistics
print(f"Median interval: {time_diffs.dt.days.median():.1f} days")
print(f"Minimum interval: {time_diffs.dt.days.min()} days")
print(f"Maximum interval: {time_diffs.dt.days.max()} days")

Average interval between pandas releases: 37.6 days
Median interval: 27.0 days
Minimum interval: 0.0 days
Maximum interval: 257.0 days


> Print the `summary` of the first (1.0.0) and second (2.0.0) major releases of pandas.

In [25]:
# Filter for the major releases 1.0.0 and 2.0.0
major_releases = pandas_releases[pandas_releases['version'].isin(['v1.0.0', 'v2.0.0'])]

# Print the summary for version 1.0.0
print("=" * 60)
print("PANDAS VERSION 1.0.0 RELEASE SUMMARY")
print("=" * 60)
version_1_summary = pandas_releases[pandas_releases['version'] == 'v1.0.0']['summary'].iloc[0]
print(version_1_summary)

print("\n" + "=" * 60)
print("PANDAS VERSION 2.0.0 RELEASE SUMMARY")
print("=" * 60)
version_2_summary = pandas_releases[pandas_releases['version'] == 'v2.0.0']['summary'].iloc[0]
print(version_2_summary)

PANDAS VERSION 1.0.0 RELEASE SUMMARY
This is a major release from 0.25.3, and includes a number of API changes, new
features, enhancements, and performance improvements along with a large number
of bug fixes.

Highlights include

* A redesigned website and documentation theme
* [Using Numba in `rolling.apply`](https://pandas.pydata.org/pandas-docs/version/1.0.0/whatsnew/v1.0.0.html#using-numba-in-rolling-apply-and-expanding-apply)
* [A new DataFrame method for converting to Markdown](https://pandas.pydata.org/pandas-docs/version/1.0.0/whatsnew/v1.0.0.html#converting-to-markdown)
* [A new scalar for missing values](https://pandas.pydata.org/pandas-docs/version/1.0.0/whatsnew/v1.0.0.html#experimental-na-scalar-to-denote-missing-values)
* Dedicated extension types for [string](https://pandas.pydata.org/pandas-docs/version/1.0.0/whatsnew/v1.0.0.html#dedicated-string-data-type) and
  [nullable boolean](https://pandas.pydata.org/pandas-docs/version/1.0.0/whatsnew/v1.0.0.html#boolean-data-typ

Let's write the dataframe to a table in DuckDB.

First, we need to import the required libraries and get the absolute path of the project.

In [26]:
import sqlalchemy as sqla
import os 

parent_dir = os.path.abspath(os.path.pardir)

In [27]:
engine = sqla.create_engine(f'duckdb:///{parent_dir}/output/unit-2-3.db') 

In [28]:
pandas_releases.to_sql("pandas_releases", engine)

-1

Let's use another resource (endpoint) now. There are many active contributors to the pandas library, and suppose we want to know how many commits each contributor has contributed. We can retrieve the data using the `stats/contributors` endpoint.

More info on the endpoint can be found [here](https://docs.github.com/en/rest/metrics/statistics?apiVersion=2022-11-28).

In [29]:
response = requests.get("https://api.github.com/repos/pandas-dev/pandas/stats/contributors", 
                        headers={"Accept": "application/vnd.github+json", "Authorization": f"Bearer {access_token}"})

In [30]:
response.status_code

200

In [31]:
contributors = response.json()

In [32]:
type(contributors)

list

In [33]:
len(contributors)

100

In [34]:
contributors[0]

{'total': 20,
 'weeks': [{'w': 1248566400, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1249171200, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1249776000, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1250380800, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1250985600, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1251590400, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1252195200, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1252800000, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1253404800, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1254009600, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1254614400, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1255219200, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1255824000, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1256428800, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1257033600, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1257638400, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1258243200, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1258848000, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1259452800, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1260057600, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1260662400, 'a': 0, 'd': 0, 'c': 0},
  {'w': 1261267200, 'a': 0, 'd':

In [35]:
contributors[0]["author"]

{'login': 'danbirken',
 'id': 91798,
 'node_id': 'MDQ6VXNlcjkxNzk4',
 'avatar_url': 'https://avatars.githubusercontent.com/u/91798?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/danbirken',
 'html_url': 'https://github.com/danbirken',
 'followers_url': 'https://api.github.com/users/danbirken/followers',
 'following_url': 'https://api.github.com/users/danbirken/following{/other_user}',
 'gists_url': 'https://api.github.com/users/danbirken/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/danbirken/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/danbirken/subscriptions',
 'organizations_url': 'https://api.github.com/users/danbirken/orgs',
 'repos_url': 'https://api.github.com/users/danbirken/repos',
 'events_url': 'https://api.github.com/users/danbirken/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/danbirken/received_events',
 'type': 'User',
 'user_view_type': 'public',
 'site_admin': False}

> Get the `total` and `login` (under `author`) fields and insert them into a dataframe. Then find out who has the most number of commits.
>
> Then, write the dataframe into a table named `pandas_contributors_commits` in duckdb.

In [38]:
# Extract the required fields from contributors data
contributors_data = []

for contributor in contributors:
    contributor_info = {
        'login': contributor['author']['login'],  # Get login from author field
        'total': contributor['total']             # Get total commits
    }
    contributors_data.append(contributor_info)

# Create a DataFrame
import pandas as pd
pandas_contributors_df = pd.DataFrame(contributors_data)

# Display the DataFrame
print("Contributors DataFrame:")
print(pandas_contributors_df.head(10))

# Find who has the most commits
top_contributor = pandas_contributors_df.loc[pandas_contributors_df['total'].idxmax()]
print(f"\nContributor with most commits:")
print(f"Login: {top_contributor['login']}")
print(f"Total commits: {top_contributor['total']}")

# Alternative way to find top contributor
print(f"\nTop 5 contributors by commits:")
top_contributors = pandas_contributors_df.nlargest(5, 'total')
print(top_contributors)

# Write to DuckDB
pandas_contributors_df.to_sql("pandas_contributors_commits", engine, if_exists='replace', index=False)
print(f"\nDataFrame successfully written to 'pandas_contributors_commits' table in DuckDB!")
print(f"Total records: {len(pandas_contributors_df)}")

Contributors DataFrame:
        login  total
0   danbirken     20
1    aflaxman     20
2  liang3zy22     21
3   AlexKirko     21
4  ryankarlos     21
5    noatamir     21
6       moink     21
7     ganevgv     22
8     minggli     22
9     Licht-T     22

Contributor with most commits:
Login: jbrockmendel
Total commits: 4885

Top 5 contributors by commits:
                 login  total
99        jbrockmendel   4885
98             jreback   4794
97                wesm   3129
96           mroeschke   1817
95  jorisvandenbossche   1562

DataFrame successfully written to 'pandas_contributors_commits' table in DuckDB!
Total records: 100


## GraphQL

Github offers a new way to access its data, which is the GraphQL API. You can read more about it here: https://docs.github.com/graphql

GraphQL is introspetive, which means you can query the schema of the API itself. This is very useful when you are trying to figure out what data you can access. You can access the schema via the GraphQL API Explorer: https://docs.github.com/en/graphql/overview/explorer and run the following query:

```graphql
{
  __schema {
    types {
      name
      kind
      description
    }
  }
}
```

Or to get the fields of a specific type (for example `"Release"`), you can run the following query:

```graphql
{
  __type(name: "Release") {
    name
    kind
    description
    fields {
      name
      description
    }
  }
}
```

It should return the following results (in JSON):

```json
{
  "data": {
    "__type": {
      "name": "Release",
      "kind": "OBJECT",
      "description": "A release contains the content for a release.",
      "fields": [
        {
          "name": "author",
          "description": "The author of the release"
        },
        {
          "name": "createdAt",
          "description": "Identifies the date and time when the object was created."
        },
        {
          "name": "databaseId",
          "description": "Identifies the primary key from the database."
        },
        {
          "name": "description",
          "description": "The description of the release."
        },
        {
          "name": "descriptionHTML",
          "description": "The description of this release rendered to HTML."
        },
        {
          "name": "id",
          "description": null
        },
        {
          "name": "isDraft",
          "description": "Whether or not the release is a draft"
        },
        {
          "name": "isLatest",
          "description": "Whether or not the release is the latest releast"
        },
        {
          "name": "isPrerelease",
          "description": "Whether or not the release is a prerelease"
        },
        {
          "name": "mentions",
          "description": "A list of users mentioned in the release description"
        },
        {
          "name": "name",
          "description": "The title of the release."
        },
        {
          "name": "publishedAt",
          "description": "Identifies the date and time when the release was created."
        },
        {
          "name": "reactionGroups",
          "description": "A list of reactions grouped by content left on the subject."
        },
        {
          "name": "reactions",
          "description": "A list of Reactions left on the Issue."
        },
        {
          "name": "releaseAssets",
          "description": "List of releases assets which are dependent on this release."
        },
        {
          "name": "repository",
          "description": "The repository that the release belongs to."
        },
        {
          "name": "resourcePath",
          "description": "The HTTP path for this issue"
        },
        {
          "name": "shortDescriptionHTML",
          "description": "A description of the release, rendered to HTML without any links in it."
        },
        {
          "name": "tag",
          "description": "The Git tag the release points to"
        },
        {
          "name": "tagCommit",
          "description": "The tag commit for this release."
        },
        {
          "name": "tagName",
          "description": "The name of the release's Git tag"
        },
        {
          "name": "updatedAt",
          "description": "Identifies the date and time when the object was last updated."
        },
        {
          "name": "url",
          "description": "The HTTP URL for this issue"
        },
        {
          "name": "viewerCanReact",
          "description": "Can user react to this subject"
        }
      ]
    }
  }
}
```

Follow the guide [here](https://docs.github.com/en/graphql/guides/introduction-to-graphql#discovering-the-graphql-api) for more info.

You can also get the schema via a `GET` request to the endpoint `https://api.github.com/graphql`:

In [39]:
response = requests.get("https://api.github.com/graphql", 
                        headers={"Authorization": f"Bearer {access_token}"})

In [40]:
schemas = response.json()

In [41]:
schemas['data']['__schema'].keys()

dict_keys(['queryType', 'mutationType', 'subscriptionType', 'types', 'directives'])

In [42]:
len(schemas['data']['__schema']['types'])

1672

In [43]:
schemas['data']['__schema']['types'][0]

{'kind': 'INPUT_OBJECT',
 'name': 'AbortQueuedMigrationsInput',
 'description': 'Autogenerated input type of AbortQueuedMigrations',
 'fields': None,
 'inputFields': [{'name': 'clientMutationId',
   'description': 'A unique identifier for the client performing the mutation.',
   'type': {'kind': 'SCALAR', 'name': 'String', 'ofType': None},
   'defaultValue': None},
  {'name': 'ownerId',
   'description': 'The ID of the organization that is running the migrations.',
   'type': {'kind': 'NON_NULL',
    'name': None,
    'ofType': {'kind': 'SCALAR', 'name': 'ID', 'ofType': None}},
   'defaultValue': None}],
 'interfaces': None,
 'enumValues': None,
 'possibleTypes': None}

It returns the same result as the `__schema` query above.

The GraphQL API only has a single endpoint:

```
https://api.github.com/graphql
```

In REST, HTTP verbs determine the operation performed. In GraphQL, you'll provide a JSON-encoded body whether you're performing a `query` or a `mutation`, so the HTTP verb is `POST`. The exception is an introspection query, which is a simple `GET` to the endpoint (which we did above).

To query GraphQL via `requests`, make a POST request with a JSON payload. The payload must contain a string called `query`.

GraphQL queries return only the data you specify. To form a query, you must specify fields within fields (also known as nested subfields) until you return only scalars.

For example, let's replicate the REST API exercise above where we extract the `releases`. You can see the resources in GraphQL which you can access [here](https://docs.github.com/en/graphql/reference/objects) - search for `releases`. 

We must specify the fields we want to extract:

In [44]:
query = """
query {
    repository(owner: "pandas-dev", name: "pandas") {
        releases(first: 100) {
            totalCount
            edges {
                node {
                    tagName
                    description
                    publishedAt
                }
            }
        }
    }
}
"""
            

response = requests.post("https://api.github.com/graphql", 
                        headers={"Authorization": f"Bearer {access_token}"}, json={"query": query})

Looking at the composition line by line:

`query {`

Because we want to read data from the server, not modify it, query is the root operation. (If you don't specify an operation, query is also the default.)

`repository(owner:"pandas-dev", name:"pandas") {`

To begin the query, we want to find a repository object. The schema validation indicates this object requires an owner and a name argument.

`releases(first: 100) {`

To account for all releases in the repository, we call the releases object. (We could query a single release on a repository, but that would require us to know the tagName of the release we want to return and provide it as an argument.)

Some details about the releases object:

The docs tell us this object has the type `ReleaseConnection`. Schema validation indicates this object requires a last or first number of results as an argument, so we provide first 100. You can find more information about the `ReleaseConnection` type [here](https://docs.github.com/en/graphql/reference/objects#releaseconnection).

- `totalCount`

    The beauty of GraphQL is that we can retrieve the totalCount of the releases object by simply adding it to the query and it will be returned.

- `edges {`

    We know releases is a connection because it has the ReleaseConnection type. To retrieve data about individual release, we have to access the node via edges.

- `nodes {`

    Here we retrieve the nodes at the end of the edge. The ReleaseConnection docs indicate the nodes at the end of the ReleaseConnection type is a Release object.

    Now that we know we're retrieving a Release object, we can look at the docs and specify the fields we want to return:
    ```
    tagName
    description
    publishedAt
    ```

    Here we specify the tagName, description, and publishedAt as before.

In [45]:
releases = response.json()

In [46]:
type(releases)

dict

In [47]:
releases

{'data': {'repository': {'releases': {'totalCount': 111,
    'edges': [{'node': {'tagName': 'v2.3.1',
       'description': 'We are pleased to announce the release of pandas 2.3.1.\r\nThis release includes some improvements and fixes to the future string data type (preview feature for the upcoming pandas 3.0). We recommend that all users upgrade to this version.\r\n\r\nSee the [full whatsnew](https://pandas.pydata.org/pandas-docs/version/2.3.1/whatsnew/v2.3.1.html) for a list of all the changes.\r\nPandas 2.3.1 supports Python 3.9 and higher.\r\n\r\nThe release will be available on the conda-forge channel:\r\n\r\n    conda install pandas --channel conda-forge\r\n\r\nOr via PyPI:\r\n\r\n    python3 -m pip install --upgrade pandas\r\n\r\nPlease report any issues with the release on the [pandas issue tracker](https://github.com/pandas-dev/pandas/issues).\r\n\r\nThanks to all the contributors who made this release possible.',
       'publishedAt': '2025-07-07T19:12:32Z'}},
     {'node': {'

In [48]:
releases['data']['repository']['releases']['totalCount']

111

In [49]:
releases['data']['repository']['releases']['edges'][0]['node']

{'tagName': 'v2.3.1',
 'description': 'We are pleased to announce the release of pandas 2.3.1.\r\nThis release includes some improvements and fixes to the future string data type (preview feature for the upcoming pandas 3.0). We recommend that all users upgrade to this version.\r\n\r\nSee the [full whatsnew](https://pandas.pydata.org/pandas-docs/version/2.3.1/whatsnew/v2.3.1.html) for a list of all the changes.\r\nPandas 2.3.1 supports Python 3.9 and higher.\r\n\r\nThe release will be available on the conda-forge channel:\r\n\r\n    conda install pandas --channel conda-forge\r\n\r\nOr via PyPI:\r\n\r\n    python3 -m pip install --upgrade pandas\r\n\r\nPlease report any issues with the release on the [pandas issue tracker](https://github.com/pandas-dev/pandas/issues).\r\n\r\nThanks to all the contributors who made this release possible.',
 'publishedAt': '2025-07-07T19:12:32Z'}

In [50]:
releases['data']['repository']['releases']['edges'][-1]['node']

{'tagName': 'v0.16.0rc1',
 'description': 'The release candidate for 0.16.0 is now previewing.\n\nThis is a major release from 0.15.2 and includes a small number of API changes, several new features, enhancements, and performance improvements along with a large number of bug fixes. We recommend that all users upgrade to this version.\n\nHighlights include:\n- `DataFrame.assign` method\n- `Series.to_coo/from_coo` methods to interact with `scipy.sparse`\n- Backwards incompatible change to `Timedelta` to conform the .seconds attribute with `datetime.timedelta`\n- Changes to the `.loc` slicing API to conform with the behavior of `.ix`\n- Changes to the default for ordering in the `Categorical` constructor\n\nSee the [Whatsnew](http://pandas-docs.github.io/pandas-docs-travis/whatsnew.html) for much more information. Please report any issues [here](https://github.com/pydata/pandas/issues)\n',
 'publishedAt': '2015-03-13T14:10:52Z'}

> Query the `issues` resource, return the first 50 issues with the fields- `title`, `createdAt` and `author`, under the `author` field, return the `login` field.
>
> Refer to this [link](https://docs.github.com/en/graphql/reference/objects#issue) for more info.

## Web Scraping

In [51]:
import requests
from bs4 import BeautifulSoup

In [52]:
r = requests.get("https://www.scrapethissite.com/pages/simple/")

In [53]:
r.status_code

200

In [54]:
type(r.text)

str

In [55]:
print(r.text)

<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Countries of the World: A Simple Example | Scrape This Site | A public sandbox for learning web scraping</title>
    <link rel="icon" type="image/png" href="/static/images/scraper-icon.png" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="description" content="A single page that lists information about all the countries in the world. Good for those just get started with web scraping.">

    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" rel="stylesheet" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" crossorigin="anonymous">
    <link href='https://fonts.googleapis.com/css?family=Lato:400,700' rel='stylesheet' type='text/css'>
    <link rel="stylesheet" type="text/css" href="/static/css/styles.css">

    
<meta name=

We will use BeautifulSoup to parse the HTML above. There are many parsers available. The default parser is `html.parser`, it does not require any additional installation. However, its performance is not the best. If you want better performance, you can use `lxml` parser. Refer to [here](https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=get#installing-a-parser) for more info.

In [56]:
soup = BeautifulSoup(r.text, "html.parser")

We can look for all paragraphs in the page using the `find_all` method. This returns a list of all the paragraph `Tag`s in the page.

Refer to the [documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#bs4.Tag) for more details.

#### Common HTML Tags you can search for:

| Tag | Description | Example Usage |
|-----|-------------|---------------|
| `'p'` | Paragraphs | `soup.find_all('p')` |
| `'h1'`, `'h2'`, `'h3'` | Headings | `soup.find_all('h1')` |
| `'div'` | Division/container | `soup.find_all('div')` |
| `'a'` | Links | `soup.find_all('a')` |
| `'img'` | Images | `soup.find_all('img')` |
| `'table'` | Tables | `soup.find_all('table')` |
| `'li'` | List items | `soup.find_all('li')` |
| `'span'` | Inline spans | `soup.find_all('span')` |

In [57]:
paragraphs = soup.find_all('p')

In [58]:
len(paragraphs)

3

In [59]:
type(paragraphs[0])

bs4.element.Tag

In [65]:
paragraphs[0]

<p class="lead">
                            A single page that lists information about all the countries in the world. Good for those just get started with web scraping.
                            Practice looking for patterns in the HTML that will allow you to extract information about each country. Then, build a simple web scraper that makes a request to this page, parses the HTML and prints out each country's name.
                        </p>

In [61]:
# get the actual text
paragraphs[0].text

"\n                            A single page that lists information about all the countries in the world. Good for those just get started with web scraping.\n                            Practice looking for patterns in the HTML that will allow you to extract information about each country. Then, build a simple web scraper that makes a request to this page, parses the HTML and prints out each country's name.\n                        "

We can find all the country names in the HTML by specifying the tag and class name. It is always good to be as specific as possible when selecting elements. This is because the more specific you are, the less likely you will select other unintended elements.

In [62]:
country_name_elements = soup.find_all('h3', 'country-name')

or equivalently

`soup.find_all('h3', class_='country-name')`

In [63]:
country_name_elements

[<h3 class="country-name">
 <i class="flag-icon flag-icon-ad"></i>
                             Andorra
                         </h3>,
 <h3 class="country-name">
 <i class="flag-icon flag-icon-ae"></i>
                             United Arab Emirates
                         </h3>,
 <h3 class="country-name">
 <i class="flag-icon flag-icon-af"></i>
                             Afghanistan
                         </h3>,
 <h3 class="country-name">
 <i class="flag-icon flag-icon-ag"></i>
                             Antigua and Barbuda
                         </h3>,
 <h3 class="country-name">
 <i class="flag-icon flag-icon-ai"></i>
                             Anguilla
                         </h3>,
 <h3 class="country-name">
 <i class="flag-icon flag-icon-al"></i>
                             Albania
                         </h3>,
 <h3 class="country-name">
 <i class="flag-icon flag-icon-am"></i>
                             Armenia
                         </h3>,
 <h3 class="countr

> Extract just the country names string into a new list called `country_names`. Remove all whitespaces.
>
> Your results should be starting with `['Andorra', 'United Arab Emirates', 'Afghanistan', 'Antigua and Barbuda', ...]`

In [66]:
country_names = []

for country_element in country_name_elements:
    country_name = country_element.text.strip()
    country_names.append(country_name)

print(country_names)

['Andorra', 'United Arab Emirates', 'Afghanistan', 'Antigua and Barbuda', 'Anguilla', 'Albania', 'Armenia', 'Angola', 'Antarctica', 'Argentina', 'American Samoa', 'Austria', 'Australia', 'Aruba', 'Åland', 'Azerbaijan', 'Bosnia and Herzegovina', 'Barbados', 'Bangladesh', 'Belgium', 'Burkina Faso', 'Bulgaria', 'Bahrain', 'Burundi', 'Benin', 'Saint Barthélemy', 'Bermuda', 'Brunei', 'Bolivia', 'Bonaire', 'Brazil', 'Bahamas', 'Bhutan', 'Bouvet Island', 'Botswana', 'Belarus', 'Belize', 'Canada', 'Cocos [Keeling] Islands', 'Democratic Republic of the Congo', 'Central African Republic', 'Republic of the Congo', 'Switzerland', 'Ivory Coast', 'Cook Islands', 'Chile', 'Cameroon', 'China', 'Colombia', 'Costa Rica', 'Cuba', 'Cape Verde', 'Curacao', 'Christmas Island', 'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Denmark', 'Dominica', 'Dominican Republic', 'Algeria', 'Ecuador', 'Estonia', 'Egypt', 'Western Sahara', 'Eritrea', 'Spain', 'Ethiopia', 'Finland', 'Fiji', 'Falkland Islands', 'Micron

> Extract the capitals into a new list called `capital_names`.

In [67]:
# Extract capital names from span tags with class 'country-capital'
capital_name_elements = soup.find_all('span', class_='country-capital')

# Extract just the text content and remove whitespaces
capital_names = [capital.text.strip() for capital in capital_name_elements]

# Display the result
print(capital_names)
print(f"\nTotal capitals found: {len(capital_names)}")
print(f"First few capitals: {capital_names[:5]}")

['Andorra la Vella', 'Abu Dhabi', 'Kabul', "St. John's", 'The Valley', 'Tirana', 'Yerevan', 'Luanda', 'None', 'Buenos Aires', 'Pago Pago', 'Vienna', 'Canberra', 'Oranjestad', 'Mariehamn', 'Baku', 'Sarajevo', 'Bridgetown', 'Dhaka', 'Brussels', 'Ouagadougou', 'Sofia', 'Manama', 'Bujumbura', 'Porto-Novo', 'Gustavia', 'Hamilton', 'Bandar Seri Begawan', 'Sucre', 'Kralendijk', 'Brasília', 'Nassau', 'Thimphu', 'None', 'Gaborone', 'Minsk', 'Belmopan', 'Ottawa', 'West Island', 'Kinshasa', 'Bangui', 'Brazzaville', 'Bern', 'Yamoussoukro', 'Avarua', 'Santiago', 'Yaoundé', 'Beijing', 'Bogotá', 'San José', 'Havana', 'Praia', 'Willemstad', 'Flying Fish Cove', 'Nicosia', 'Prague', 'Berlin', 'Djibouti', 'Copenhagen', 'Roseau', 'Santo Domingo', 'Algiers', 'Quito', 'Tallinn', 'Cairo', 'Laâyoune / El Aaiún', 'Asmara', 'Madrid', 'Addis Ababa', 'Helsinki', 'Suva', 'Stanley', 'Palikir', 'Tórshavn', 'Paris', 'Libreville', 'London', "St. George's", 'Tbilisi', 'Cayenne', 'St Peter Port', 'Accra', 'Gibraltar', '

It is common to collect all the required information at one go. For each country element, you can further use the `find` method to extract the child element.

Let's collect the capital, population and area for each country.

In [68]:
countries = []
for country in soup.find_all('div', 'col-md-4 country'):
    country_info = {}
    country_info['name'] = country.find('h3').text.strip()
    country_info['capital'] = country.find('span', 'country-capital').text
    country_info['population'] = country.find('span', 'country-population').text
    country_info['area_km2'] = country.find('span', 'country-area').text
    countries.append(country_info)

Let's scrape a more complex example. This page has a table with pagination. We will scrape the rows in all the pages and store the data in a dataframe.

[https://www.scrapethissite.com/pages/forms/](https://www.scrapethissite.com/pages/forms/)

First, as with any web scraping task, we need to inspect the page to understand the structure of the page.

We'll start by scraping the first page:

In [69]:
r = requests.get("https://www.scrapethissite.com/pages/forms/")

In [70]:
r.status_code

200

In [71]:
soup = BeautifulSoup(r.text, "html.parser")

Let's extract the table header first:

In [72]:
header = soup.find('tr')

In [73]:
header

<tr>
<th>
                            Team Name
                        </th>
<th>
                            Year
                        </th>
<th>
                            Wins
                        </th>
<th>
                            Losses
                        </th>
<th>
                            OT Losses
                        </th>
<th>
                            Win %
                        </th>
<th>
                            Goals For (GF)
                        </th>
<th>
                            Goals Against (GA)
                        </th>
<th>
                            + / -
                        </th>
</tr>

In [74]:
headers = [th.text.strip() for th in header.find_all('th')]

In [75]:
headers

['Team Name',
 'Year',
 'Wins',
 'Losses',
 'OT Losses',
 'Win %',
 'Goals For (GF)',
 'Goals Against (GA)',
 '+ / -']

Let's find all the rows on first page:

In [76]:
teams = soup.find_all('tr', 'team')

In [77]:
teams[0]

<tr class="team">
<td class="name">
                            Boston Bruins
                        </td>
<td class="year">
                            1990
                        </td>
<td class="wins">
                            44
                        </td>
<td class="losses">
                            24
                        </td>
<td class="ot-losses">
</td>
<td class="pct text-success">
                            0.55
                        </td>
<td class="gf">
                            299
                        </td>
<td class="ga">
                            264
                        </td>
<td class="diff text-success">
                            35
                        </td>
</tr>

In [78]:
for col in teams[0].find_all('td'):
    print(col.text.strip())

Boston Bruins
1990
44
24

0.55
299
264
35


We can combine that with the header:

In [79]:
row_dict = {}
for header, col in zip(headers, teams[0].find_all('td')):
    row_dict[header] = col.text.strip()

In [80]:
row_dict

{'Team Name': 'Boston Bruins',
 'Year': '1990',
 'Wins': '44',
 'Losses': '24',
 'OT Losses': '',
 'Win %': '0.55',
 'Goals For (GF)': '299',
 'Goals Against (GA)': '264',
 '+ / -': '35'}

> Repeat the operation above for all the teams (rows) using a `for` loop, and append each row_dict to a list called `rows`.

In [81]:
# Create an empty list to store all team data
rows = []

# Loop through all team rows
for team in teams:  # teams was defined earlier as soup.find_all('tr', 'team')
    row_dict = {}
    
    # Extract data from each column using zip
    for header, col in zip(headers, team.find_all('td')):
        row_dict[header] = col.text.strip()
    
    # Append this team's data to the rows list
    rows.append(row_dict)

# Display results
print(f"Total teams extracted: {len(rows)}")
print("First team:", rows[0])
print("Last team:", rows[-1])

Total teams extracted: 25
First team: {'Team Name': 'Boston Bruins', 'Year': '1990', 'Wins': '44', 'Losses': '24', 'OT Losses': '', 'Win %': '0.55', 'Goals For (GF)': '299', 'Goals Against (GA)': '264', '+ / -': '35'}
Last team: {'Team Name': 'Chicago Blackhawks', 'Year': '1991', 'Wins': '36', 'Losses': '29', 'OT Losses': '', 'Win %': '0.45', 'Goals For (GF)': '257', 'Goals Against (GA)': '236', '+ / -': '21'}


If you press on any page number (button) in the bottom, you will see the URL change, for example pressing on page 2:

`https://www.scrapethissite.com/pages/forms/?page_num=2`

This is because the page number is a parameter in the URL. We can use this to our advantage to scrape all the pages. We can use a `for loop` to iterate through all the pages and scrape the data from each page.

In [82]:

def parse_and_extract_rows(response_text: str):
    """
    Parse the HTML and extract table rows from the response object's text.
    
    Args:
        response_text (str): The HTML text from the response object.
        
    Returns:
        An iterator of dictionaries with the data from the current page.
    """
    soup = BeautifulSoup(response_text, "html.parser")
    header = soup.find('tr')
    headers = [th.text.strip() for th in header.find_all('th')]
    teams = soup.find_all('tr', 'team')
    for team in teams:
        row_dict = {}
        for header, col in zip(headers, team.find_all('td')):
            row_dict[header] = col.text.strip()
        yield row_dict

As there are 24 pages, we need to iterate through all of them to get all the data we want. We will use a for loop to do this. We will also use the `time.sleep()` function to pause and make sure we don't overload the server with requests.

In [83]:
import time

In [84]:
rows = []
for page in range(1, 25):
    r = requests.get(f"https://www.scrapethissite.com/pages/forms/?page_num={page}")
    for row_dict in parse_and_extract_rows(r.text):
        rows.append(row_dict)
    # pause for 1 second between requests
    time.sleep(1)

In [85]:
len(rows)

582

In [86]:
rows[0]

{'Team Name': 'Boston Bruins',
 'Year': '1990',
 'Wins': '44',
 'Losses': '24',
 'OT Losses': '',
 'Win %': '0.55',
 'Goals For (GF)': '299',
 'Goals Against (GA)': '264',
 '+ / -': '35'}

In [87]:
rows[-1]

{'Team Name': 'Winnipeg Jets',
 'Year': '2011',
 'Wins': '37',
 'Losses': '35',
 'OT Losses': '10',
 'Win %': '0.451',
 'Goals For (GF)': '225',
 'Goals Against (GA)': '246',
 '+ / -': '-21'}

Let's convert the list of dictionaries into a dataframe to perform further analysis.

In [88]:
df = pd.DataFrame(rows)

 Cast the correct dtypes for the columns, we'll use pandas [nullable](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes) integer (`pd.Int64Dtype`) and float (`pd.Float64Dtype`) type for the numeric columns.

 Below, `int` is an alias for `np.int64`, while `Int64` represents `pd.Int64Dtype`.

In [89]:
df = df.replace('', pd.NA).astype({'Year': int, 'Wins': 'Int64', 'Losses': 'Int64', 'OT Losses': 'Int64', 
                                   'Goals For (GF)': 'Int64', 'Goals Against (GA)': 'Int64', 
                                   '+ / -': 'Int64', 'Win %': float})

In [90]:
df

,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,<NA>,0.550,299,264,35
1,Buffalo Sabres,1990,31,30,<NA>,0.388,292,278,14
2,Calgary Flames,1990,46,26,<NA>,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,<NA>,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,<NA>,0.425,273,298,-25
...,...,...,...,...,...,...,...,...,...
577,Tampa Bay Lightning,2011,38,36,8,0.463,235,281,-46
578,Toronto Maple Leafs,2011,35,37,10,0.427,231,264,-33
579,Vancouver Canucks,2011,51,22,9,0.622,249,198,51
580,Washington Capitals,2011,42,32,8,0.512,222,230,-8


> 1. Find the team that has the most number of Wins for the year.
> 
> 2. Find the team that has the most number of Losses for the year.
> 
> 3. Find the team that has the lowest Win % for the year.
> 
> 4. Find the team that has the most number of Wins for all years.

In [91]:
# Find team with maximum wins
max_wins_team = df.loc[df['Wins'].idxmax()]
print("Team with most wins:")
print(f"Team: {max_wins_team['Team Name']}")
print(f"Year: {max_wins_team['Year']}")
print(f"Wins: {max_wins_team['Wins']}")

Team with most wins:
Team: Detroit Red Wings
Year: 1995
Wins: 62


In [95]:
# Find team with maximum losses
max_losses_team = df.loc[df['Losses'].idxmax()]
print("Team with most losses:")
print(f"Team: {max_losses_team['Team Name']}")
print(f"Year: {max_losses_team['Year']}")
print(f"Losses: {max_losses_team['Losses']}")

Team with most losses:
Team: San Jose Sharks
Year: 1992
Losses: 71


In [96]:
# Find team with minimum win percentage
min_winpct_team = df.loc[df['Win %'].idxmin()]
print("Team with lowest Win %:")
print(f"Team: {min_winpct_team['Team Name']}")
print(f"Year: {min_winpct_team['Year']}")
print(f"Win %: {min_winpct_team['Win %']}")

Team with lowest Win %:
Team: Ottawa Senators
Year: 1992
Win %: 0.119


In [97]:
# Group by team name and sum all wins across years
team_total_wins = df.groupby('Team Name')['Wins'].sum().sort_values(ascending=False)

# Get the team with most total wins
most_wins_overall = team_total_wins.index[0]
total_wins = team_total_wins.iloc[0]

print("Team with most wins across all years:")
print(f"Team: {most_wins_overall}")
print(f"Total Wins: {total_wins}")

# Show top 5 teams for context
print("\nTop 5 teams by total wins:")
print(team_total_wins.head())

Team with most wins across all years:
Team: Detroit Red Wings
Total Wins: 986

Top 5 teams by total wins:
Team Name
Detroit Red Wings      986
New Jersey Devils      905
Pittsburgh Penguins    839
Philadelphia Flyers    836
Boston Bruins          816
Name: Wins, dtype: Int64


In [ ]:
def parse_and_extract_rows(soup: BeautifulSoup):
    """
    Extract table rows from the parsed HTML.
    
    Args:
        soup: The parsed HTML.
        
    Returns:
        An iterator of dictionaries with the data from the current page.
    """
    header = soup.find('tr')
    headers = [th.text.strip() for th in header.find_all('th')]
    teams = soup.find_all('tr', 'team')
    for team in teams:
        row_dict = {}
        for header, col in zip(headers, team.find_all('td')):
            row_dict[header] = col.text.strip()
        yield row_dict

In [ ]:
rows = []
page = 1
r = requests.get(f"https://www.scrapethissite.com/pages/forms/?page_num={page}")
soup = BeautifulSoup(r.text, "html.parser")
for row_dict in parse_and_extract_rows(soup):
    rows.append(row_dict)

while soup.find("a", {"aria-label": "Next"}):
    page += 1
    r = requests.get(f"https://www.scrapethissite.com/pages/forms/?page_num={page}")
    soup = BeautifulSoup(r.text, "html.parser")
    for row_dict in parse_and_extract_rows(soup):
        rows.append(row_dict)
    # pause for 1 second between requests
    time.sleep(1)

In [ ]:
len(rows)

In [ ]:
rows[0]

In [ ]:
rows[-1]